# Processing of neuronal data

This notebook guides through the process of populating tables found in `neuro_data.static_images` following experiments with static images. These steps are necessary to be able to train NN on this data.

In [1]:
from neuro_data.static_images import data_schemas as data, stats, configs
from neuro_data.static_images.data_schemas import meso, fuse, anatomy, stimulus, experiment
from staticnet_analyses import multi_mei, closed_loop

import datajoint as dj
from time import time
import numpy as np
from itertools import compress

dj.config['display.limit'] = 30

Connecting ecobost@at-database.ad.bcm.edu:3306


/src/static-networks/staticnet_analyses/multi_mei.py:1649: UserWarning: Use of this table is deprecated. It is kept only for record keeping purpose
  warnings.warn('Use of this table is deprecated. It is kept only for record keeping purpose')


# Listing of MEI scans

All MEI related scans are listed in the variable `MEI_STATIC`. **Add new MEI scans into the variable `MEI_STATIC`**.

In [2]:
data.MEI_STATIC

['(animal_id=20505 AND session=2 AND scan_idx=24)',
 '(animal_id=20505 AND session=3 AND scan_idx=7)',
 '(animal_id=20505 AND session=5 AND scan_idx=26)',
 '(animal_id=20505 AND session=6 AND scan_idx=1)',
 '(animal_id=20505 AND session=7 AND scan_idx=23)',
 '(animal_id=20505 AND session=7 AND scan_idx=29)',
 '(animal_id=20457 AND session=5 AND scan_idx=9)',
 '(animal_id=20457 AND session=5 AND scan_idx=17)',
 '(animal_id=20457 AND session=5 AND scan_idx=27)',
 '(animal_id=20457 AND session=7 AND scan_idx=10)',
 '(animal_id=20457 AND session=7 AND scan_idx=16)',
 '(animal_id=20457 AND session=8 AND scan_idx=9)',
 '(animal_id=20457 AND session=8 AND scan_idx=12)',
 '(animal_id=20457 AND session=8 AND scan_idx=22)',
 '(animal_id=20505 AND session=10 AND scan_idx=14)',
 '(animal_id=20505 AND session=10 AND scan_idx=19)',
 '(animal_id=20505 AND session=11 AND scan_idx=16)',
 '(animal_id=20505 AND session=12 AND scan_idx=16)',
 '(animal_id=20505 AND session=12 AND scan_idx=29)',
 '(animal_i

# Exclude bad trials

Find trials with improper number of flip times

In [7]:
targets = '(animal_id=22620 AND session=4 AND scan_idx=15)'

In [8]:
fuse.ScanDone & targets

animal_id id number,session session index for the mouse,scan_idx number of TIFF stack file,pipe_version,segmentation_method,spike_method spike inference method,pipe pipeline name
22620,4,15,1,6,5,meso


In [9]:
unprocessed = (fuse.ScanDone & targets) - data.InputResponse

In [10]:
unprocessed

animal_id id number,session session index for the mouse,scan_idx number of TIFF stack file,pipe_version,segmentation_method,spike_method spike inference method,pipe pipeline name
22620,4,15,1,6,5,meso


In [11]:
keys, flips = (stimulus.Trial & stimulus.Frame & unprocessed - data.ExcludedTrial).fetch('KEY', 'flip_times')

In [12]:
n_flips = np.array([f.shape[1] for f in flips])

In [13]:
len(n_flips)

6000

In [14]:
n_flips

array([3, 3, 3, ..., 3, 3, 3])

There should only be 3 frames for `stimulus.Frame` stimulus.

In [15]:
correct_flips = 3

In [16]:
bad_trials = (n_flips != correct_flips)
sum(bad_trials)

0

In [17]:
bad_trial_keys = list(compress(keys, bad_trials))

In [18]:
data.ExcludedTrial.insert(bad_trial_keys)

# Processing of the data

**NOTE:** All of the following steps can be and should be completed by invoking `closed_loop.fill_data(cond)`. In particular, filling of `ConditionTier` and `InputResponse` should **not** be performed in a notebook due to long output.

Add the new scan information into `MEI_STATIC` variable defined in `data_schemas.py`

In [19]:
%%time
data.StaticScan().populate(targets)

CPU times: user 64.4 ms, sys: 9.08 ms, total: 73.4 ms
Wall time: 460 ms


In [20]:
%%time
data.ConditionTier().populate(targets)

08-10-2019:03:11:56 INFO     data_schemas.py      337:	 --------------------------------------------------------------------------------
08-10-2019:03:11:56 INFO     data_schemas.py      338:	 Processing {'animal_id': 22620,
 'pipe_version': 1,
 'scan_idx': 15,
 'segmentation_method': 6,
 'session': 4,
 'spike_method': 5}
08-10-2019:03:11:56 INFO     data_schemas.py      349:	 Inserting assignment from ImageNetSplit
Inserting 5100 imagenet conditions!
CPU times: user 69 ms, sys: 5.95 ms, total: 74.9 ms
Wall time: 1.64 s


In [21]:
%%time
data.Frame.populate('preproc_id=0')

CPU times: user 21.2 ms, sys: 1.3 ms, total: 22.5 ms
Wall time: 2.48 s


In [22]:
%%time
data.InputResponse().populate(targets, 'preproc_id=0')

08-10-2019:03:12:03 INFO     data_schemas.py      578:	 Sampling neural responses at 0.5s intervals
08-10-2019:03:12:30 INFO     data_schemas.py      543:	 Loaded 7957 traces
08-10-2019:03:12:30 INFO     data_schemas.py      545:	 Generating lowpass filters to 2.0Hz
Using separate time traces per spline
08-10-2019:03:13:21 INFO     data_schemas.py      582:	 Excluding 0 trials based on ExcludedTrial
08-10-2019:03:14:56 INFO     data_schemas.py      558:	 Found 3 flip times
08-10-2019:03:14:56 INFO     data_schemas.py      598:	 Sampling 6000 responses 0.3s after stimulus onset


Computing splines: 100%|██████████| 7957/7957 [04:58<00:00, 26.68it/s]


CPU times: user 6min 12s, sys: 15.6 s, total: 6min 27s
Wall time: 8min 12s


In [23]:
%%time
data.Eye().populate(targets)

08-10-2019:03:20:16 INFO     data_schemas.py      844:	 Populating {'animal_id': 22620,
 'pipe_version': 1,
 'preproc_id': 0,
 'scan_idx': 15,
 'segmentation_method': 6,
 'session': 4,
 'spike_method': 5,
 'tracking_method': 2}
08-10-2019:03:20:24 INFO     data_schemas.py      800:	 Keeping some nans in the pupil location trace
08-10-2019:03:20:24 INFO     data_schemas.py      803:	 Keeping some nans in the pupil radius trace
08-10-2019:03:20:24 INFO     data_schemas.py      809:	 Loading behavior frametimes


/src/neuro_data/neuro_data/static_images/data_schemas.py:798: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  xy = np.vstack(map(partial(fill_nans, preserve_gap=3), xy.T))


08-10-2019:03:20:24 INFO     data_schemas.py      861:	 Downsampling eye signal to 2.0Hz
Using one common time traces per spline
08-10-2019:03:20:25 INFO     data_schemas.py      558:	 Found 3 flip times
08-10-2019:03:20:25 WARNING  data_schemas.py      891:	 Found 22 NaN trials. Setting to -1
CPU times: user 9.59 s, sys: 139 ms, total: 9.72 s
Wall time: 10 s


In [24]:
%%time
data.Treadmill().populate(targets)

08-10-2019:03:20:26 INFO     data_schemas.py      917:	 Populating
{'animal_id': 22620,
 'pipe_version': 1,
 'preproc_id': 0,
 'scan_idx': 15,
 'segmentation_method': 6,
 'session': 4,
 'spike_method': 5}
08-10-2019:03:20:26 INFO     data_schemas.py      809:	 Loading behavior frametimes
08-10-2019:03:20:26 INFO     data_schemas.py      933:	 Downsampling treadmill signal to 2.0Hz
08-10-2019:03:20:26 WARNING  data.py               47:	 Found nans in the x-values. Replacing them with linear interpolation
08-10-2019:03:20:26 INFO     data_schemas.py      558:	 Found 3 flip times
CPU times: user 1.32 s, sys: 36.8 ms, total: 1.36 s
Wall time: 1.59 s


## Create Static Dataset
Update `StaticMultiDataset`'s `selection` list inside `fill` method to add new scans/preprocessing combination.

In [25]:
from importlib import reload
reload(data)

<module 'neuro_data.static_images.data_schemas' from '/src/neuro_data/neuro_data/static_images/data_schemas.py'>

In [26]:
data.StaticMultiDataset().fill()

Already found entry {'group_id': 0, 'description': '11521-7-1'}
Already found entry {'group_id': 1, 'description': '11521-7-2'}
Already found entry {'group_id': 2, 'description': '16157-5-5'}
Already found entry {'group_id': 3, 'description': '16157-5-6'}
Already found entry {'group_id': 4, 'description': '16157-5-5-scaled'}
Already found entry {'group_id': 5, 'description': '16312-3-20'}
Already found entry {'group_id': 6, 'description': '11521-7-1-scaled'}
Already found entry {'group_id': 7, 'description': '11521-7-2-scaled'}
Already found entry {'group_id': 8, 'description': '18765-4-6'}
Already found entry {'group_id': 9, 'description': '16157-5'}
Already found entry {'group_id': 10, 'description': '20505-2-24'}
Already found entry {'group_id': 11, 'description': '20505-3-7'}
Already found entry {'group_id': 12, 'description': '20505-6-1'}
Already found entry {'group_id': 13, 'description': '20505-7-29'}
Already found entry {'group_id': 14, 'description': '20457-5-9'}
Already found

## (Deprecated) Fill Area and Layer map with V1 L2/3

Most MEI scans have been performed on V1 L2/3. Check if True.

In [27]:
missing_scans = (meso.ScanDone & targets) - (anatomy.AreaMembership * anatomy.LayerMembership)

In [28]:
missing_scans

animal_id id number,session session index for the mouse,scan_idx number of TIFF stack file,pipe_version,segmentation_method,spike_method spike inference method
22620,4,15,1,6,5


In [29]:
units = (meso.ScanSet.Unit & missing_scans).fetch('KEY')

In [30]:
for u in units:
    u['brain_area'] = 'V1'
    u['layer'] = 'L2/3'

In [31]:
anatomy.AreaMembership.insert(units, ignore_extra_fields=True, allow_direct_insert=True, skip_duplicates=True)

In [32]:
anatomy.LayerMembership.insert(units, ignore_extra_fields=True, allow_direct_insert=True, skip_duplicates=True)